# NeMo Curator - Synthetic Data Generation (SDG)

## 生成開放式問題
依照以下步驟生成開放式問題：
1. 生成一份關於宏觀的主題列表。
2. 生成與每個主題相關的子主題列表。
3. 生成與子主題相關的問題集。
4. 重新修正問題以更詳細。

您可以按以下的範例程式碼，使用`NeMo Curator`執行每個步驟：

In [1]:
import asyncio

from openai import AsyncOpenAI
from nemo_curator import AsyncOpenAIClient
from nemo_curator.synthetic import AsyncNemotronGenerator

from prompts import (
    DEFAULT_MACRO_TOPICS_PROMPT_TEMPLATE,
    DEFAULT_SUBTOPICS_PROMPT_TEMPLATE,
    DEFAULT_OPEN_QA_FROM_TOPICS_PROMPT_TEMPLATE,
    DEFAULT_REVISE_OPEN_QA_PROMPT_TEMPLATE
)

import nest_asyncio
nest_asyncio.apply()

/usr/local/lib/python3.12/dist-packages/optuna/study/_optimize.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [2]:
openai_client = AsyncOpenAI(
    base_url="http://localhost:8000/v1/",
    api_key="token-abc123"
)
client = AsyncOpenAIClient(openai_client)

generator = AsyncNemotronGenerator(client, max_concurrent_requests=128)

In [3]:
model = "engine"
model_kwargs = {
    "temperature": 0.2,
    "top_p": 0.7,
    "max_tokens": 1024,
}

In [4]:
async def generate(gen_function, item_list, **kwarg):
    tasks = [
        gen_function(
            item, **kwarg
        )
        for item in item_list
    ]
    responses = await asyncio.gather(*tasks, return_exceptions=True)

    results = []
    for res in responses:
        if isinstance(res, Exception):
            continue
        
        results.extend(res)

    return results

### 生成一份關於宏觀的主題列表

In [5]:
N_MACRO_TOPICS = 5

macro_topics_responses = await generator.generate_macro_topics(
    n_macro_topics=N_MACRO_TOPICS, model=model,model_kwargs=model_kwargs,
    prompt_template=DEFAULT_MACRO_TOPICS_PROMPT_TEMPLATE,
)

macro_topics_list = asyncio.run(
    generate(
        generator.convert_response_to_yaml_list, 
        macro_topics_responses, model=model, model_kwargs=model_kwargs,
    )
)

for i, macro_topics in enumerate(macro_topics_list):
    print(f"Macro Topics {i+1}:", macro_topics)

Macro Topics 1: 食物和飲料。
Macro Topics 2: 科技與未來生活。
Macro Topics 3: 環境保護與可持續發展。
Macro Topics 4: 世界文化與旅遊。
Macro Topics 5: 人體健康與醫學進展。


### 生成與每個主題相關的子主題列表

In [6]:
N_SUBTOPICS = 5

subtopic_responses = asyncio.run(
    generate(
        generator.generate_subtopics, 
        item_list=macro_topics_list, n_subtopics=N_SUBTOPICS, model=model, model_kwargs=model_kwargs,
        prompt_template=DEFAULT_SUBTOPICS_PROMPT_TEMPLATE,
    )
)

subtopic_list = asyncio.run(
    generate(
        generator.convert_response_to_yaml_list,
        subtopic_responses, model=model, model_kwargs=model_kwargs,
    )
)

for i, subtopic in enumerate(subtopic_list):
    print(f"Subtopic {i+1}:", subtopic)

Subtopic 1: 全球美食探索：從街頭小吃到高級料理
Subtopic 2: 健康飲食與營養學：現代生活中的平衡選擇
Subtopic 3: 食品科技與創新：從實驗室到餐桌
Subtopic 4: 飲食文化與社會影響：歷史、藝術與傳承
Subtopic 5: 可持續飲食與環境保護：綠色選擇與未來趨勢
Subtopic 6: 智能城市與可持續發展：探索未來城市的智能交通、能源管理與環境保護技術
Subtopic 7: 虛擬現實與增強現實在教育中的應用：重塑學習體驗與知識獲取方式
Subtopic 8: 人工智能在醫療健康領域的突破：從精準診斷到個性化治療
Subtopic 9: 未來工作場景：自動化、遠程工作與職業技能的轉型
Subtopic 10: 量子計算與未來通信：探索下一代計算技術及其對社會的影響
Subtopic 11: 城市綠化與生態建築：探索綠色建築設計與城市綠化策略，促進城市環境的可持續發展。
Subtopic 12: 循環經濟與資源回收：研究如何通過循環經濟模式，提高資源利用率，減少浪費，促進環境保護。
Subtopic 13: 可再生能源的未來：分析太陽能、風能等可再生能源的技術進展與應用前景，及其對全球能源結構轉型的影響。
Subtopic 14: 生物多樣性保護與生態平衡：探討生物多樣性的重要性，以及如何通過保護措施維持生態系統的健康與穩定。
Subtopic 15: 氣候變化與適應策略：評估氣候變化的影響，並提出有效的適應和減緩策略，以保護自然環境和人類社會。
Subtopic 16: 世界遺產之旅：探索人類文明的瑰寶
Subtopic 17: 美食與文化：舌尖上的世界之旅
Subtopic 18: 節慶與傳統：全球節日慶典大賞
Subtopic 19: 自然奇觀與生態旅遊：地球的隱藏寶藏
Subtopic 20: 現代與傳統的交融：城市文化探索
Subtopic 21: 基因編輯技術在遺傳性疾病治療中的應用與倫理考量
Subtopic 22: 數位健康與遠距醫療：未來的健康照護模式
Subtopic 23: 營養學與慢性疾病預防：最新研究與實踐指南
Subtopic 24: 免疫療法在癌症治療中的突破與挑戰
Subtopic 25: 心理健康與社會福祉：跨學科方法在預防與治療中的作用


### 生成與子主題相關的問題集

In [7]:
N_OPENLINES = 10

question_responses = asyncio.run(
    generate(
        generator.generate_open_qa_from_topic, 
        subtopic_list, n_openlines=N_OPENLINES, model=model, model_kwargs=model_kwargs,
        prompt_template=DEFAULT_OPEN_QA_FROM_TOPICS_PROMPT_TEMPLATE,
    )
)

questions = asyncio.run(
    generate(
        generator.convert_response_to_yaml_list,
        question_responses, model=model, model_kwargs=model_kwargs,
    )
)

for i, question in enumerate(questions):
    print(f"Question {i+1}:", question)

Question 1: 你能介紹一些全球各地著名的街頭小吃，並說明它們的特色和來源嗎？
Question 2: 世界各國有哪些獨特的高級料理，它們通常使用哪些珍稀食材？
Question 3: 有哪些國家的美食文化特別強調季節性食材的使用，你能舉例說明嗎？
Question 4: 你能分享一些全球知名的美食節慶活動，以及這些活動中必試的美食嗎？
Question 5: 世界各地有哪些傳統飲料，它們是如何與當地的美食相搭配的？
Question 6: 你能介紹一些融合了多國料理特色的餐廳或菜式，並說明這種融合是如何發生的？
Question 7: 有哪些國家的料理特別注重視覺呈現，你能分享一些例子嗎？
Question 8: 你能探討一下全球美食趨勢，例如植物基食品的崛起，對傳統料理的影響？
Question 9: 世界各地有哪些獨特的用餐儀式或禮節，它們是如何反映當地文化的？
Question 10: 你能提供一些探索全球美食的最佳旅行目的地，以及在這些地方必試的當地美食嗎？
Question 11: 現代生活中，如何在忙碌的工作與生活節奏中維持均衡的飲食？
Question 12: 請分享一些簡單易做的營養早餐食譜，適合早晨時間緊迫的上班族。
Question 13: 現代人常外食，如何選擇較健康的餐點，以確保營養均衡？
Question 14: 在外就餐時，有哪些技巧可以幫助減少油脂和鹽分的攝取？
Question 15: 現代生活中的壓力如何影響我們的飲食選擇，以及如何改善？
Question 16: 請提供一些適合現代人快速生活節奏的健康零食選擇。
Question 17: 如何利用現代科技（如手機應用程式）來幫助追蹤和改善飲食習慣？
Question 18: 現代人常有睡眠不足的問題，這對飲食和營養吸收有什麼影響？
Question 19: 在現代生活中，如何平衡植物性與動物性蛋白質的攝取，以促進健康？
Question 20: 請討論現代飲食趨勢，如無麩質、低碳水化合物飲食等，對健康有何影響？
Question 21: 食品科技如何利用基因編輯技術改善作物的營養價值和抗病能力？
Question 22: 從實驗室到餐桌，食品科技在減少食物浪費方面有哪些創新的解決方案？
Question 23: 人工肉類的開發過程是什麼，它如何影響未來的食品供應和環境保護

### 重新修正問題以更詳細

In [8]:
N_REVISIONS = 5

revised_questions_responses = asyncio.run(
    generate(
        generator.revise_open_qa,
        questions, n_revisions=N_REVISIONS, model=model, model_kwargs=model_kwargs,
        prompt_template=DEFAULT_REVISE_OPEN_QA_PROMPT_TEMPLATE,
    )
)

revised_questions = asyncio.run(
    generate(
        generator.convert_response_to_yaml_list,
        revised_questions_responses, model=model, model_kwargs=model_kwargs,
    )
)

for i, revised_question in enumerate(revised_questions):
    print(f"Revised Question {i+1}:", revised_question)

Revised Question 1: 街頭小吃不僅是城市文化的縮影，也是旅行者探索當地風味的窗口。你能介紹一些全球各地著名的街頭小吃，並說明它們的特色和來源嗎？
Revised Question 2: 作為一個美食愛好者，我對世界各地的街頭小吃非常感興趣。請列出五種來自不同國家的著名街頭小吃，並簡要描述它們的特色和起源。
Revised Question 3: 街頭小吃是許多城市不可或缺的一部分，它們往往蘊含著豐富的文化和歷史故事。請選擇三個你認為最具代表性的街頭小吃，詳細介紹它們的特色、來源以及在当地文化中的地位。
Revised Question 4: 世界各地的街頭小吃不僅美味，還能反映當地的飲食文化和生活習俗。請介紹一些你認為最獨特的街頭小吃，並解釋它們為什麼特別。
Revised Question 5: 街頭小吃是旅行中不可或缺的體驗之一，它們往往能帶給遊客最真實的當地風味。請分享一些你認為旅行者必試的全球著名街頭小吃，並簡述它們的特色和來源。
Revised Question 6: 世界各地的美食文化豐富多樣，每種高級料理都蘊含著當地的歷史和文化特色。請介紹一些世界各國獨特的高級料理，以及這些料理中常用的珍稀食材，並探討這些食材如何影響了當地的飲食文化。
Revised Question 7: 世界各國的高級料理往往使用一些珍稀食材，這些食材不僅增添了料理的風味，也反映了當地的自然資源和文化特色。請列出五種來自不同國家的高級料理及其使用的珍稀食材，並簡要說明這些食材的獨特之處。
Revised Question 8: 世界各國的高級料理中，有哪些使用了珍稀食材，這些食材的採集或生產過程有何特殊之處？請詳細描述三種這樣的料理及其食材，並討論這些食材對料理的影響。
Revised Question 9: 探索世界各國的高級料理，了解其使用的珍稀食材，可以幫助我們更好地認識不同文化的飲食習慣和自然環境。請選擇三個不同地區的高級料理，介紹其獨特的珍稀食材，並分析這些食材如何體現了當地的自然和文化特色。
Revised Question 10: 世界各國的高級料理中，珍稀食材的使用不僅增添了料理的風味，也往往帶有濃厚的文化和歷史意義。請選擇一種你最感興趣的高級料理，詳細介紹其使用的珍稀食材，以及這些食材在料理中的作用和文化意義。
Revised Question